In [1]:
import os, math
import pandas as pd
import numpy as np
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:

from gensim.models import Word2Vec as w2v
from os import path as osp

root = osp.dirname(os.getcwd())
print(root)
model = w2v.load(osp.join(root,'TM Project/stackexchange_model'))

unable to import 'smart_open.gcs', disabling that module


C:\Users\priya


In [3]:
def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [4]:

cvs = pd.read_csv('prc_data.csv', sep='\t')
print(cvs)
cvs = cvs.set_index('Unnamed: 0')

  Unnamed: 0                                              cv100  \
0        edu  good Knowledge b2b b2c Education Domain . \n e...   
1        exp  work experience \n software developer \n Vspac...   
2      skill  seek position utilize Software development ski...   
3      extra  Amit ekbote \n Software Developer - Vspace sof...   

                                               cv101  \
0  education \n RTMNU Priyadarshani College Engin...   
1  â¢ ability work pressure , stress , meet dead...   
2  â¢ have good knowledge application developmen...   
3  Sonu Sahu \n Nagpur , Maharashtra \n software ...   

                                               cv102  \
0                                                NaN   
1  â¢ work TDD ( Test Driven Development ) Jasmi...   
2  â¢ strong experience Twitter Bootstrap 3 fram...   
3  Jay Kavathiya \n - end Developer - General Ele...   

                                               cv103  \
0  education \n National Institute Electronics

In [5]:

jd = pd.read_csv(os.getcwd()+'\jd.csv')
print(jd['description'])


0    a leading ecommerce agency is looking to hire ...
Name: description, dtype: object


In [6]:
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(mod_word)

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.

In [7]:
for description in jd['description']:
    prc_description = ''
    for word in description.split():
        mod_word = modify(word)
        if (mod_word is not None):
            prc_description += (mod_word + ' ')
    print(prc_description)
    break

lead ecommerce agency look hire web developer join team ecommerce developer role involve work different type client candidate ability create implement unique engage intuitive work meet client requirement requirement 34 year experience web development year ecommerce project agency set development skill follow technology o aspnet asp o vb vbnet o sql server tsql o comdcom o xml html o javascript knowledge microsoft commerce server ecommerce solution sound like challenge ready send cv 


In [8]:
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print(similar_words[i], word_value[similar_words[i]])

C:\Users\priya\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


lead 1
cause 0.6887490749359131
induce 0.630913496017456
ecommerce 1
virtuemart 0.5609263181686401
marketplace 0.5272524952888489
agency 1
government 0.6580805778503418
agencies 0.6089067459106445
look 1
looked 0.6417942643165588
peek 0.5807271599769592
hire 1
recruit 0.6553700566291809
supervise 0.5519425272941589
web 1
webpage 0.5117195248603821
website 0.4991326332092285
developer 1
developers 0.5717995166778564
programmer 0.5414242744445801
join 1
joining 0.6186106204986572
joined 0.5738263726234436
team 1
teams 0.6584339141845703
crew 0.519210934638977
ecommerce 2
virtuemart 1.1218526363372803
marketplace 1.0545049905776978
developer 2
developers 1.143599033355713
programmer 1.0828485488891602
role 1
roles 0.6746314764022827
minston 0.4418644309043884
involve 1
require 0.7514784336090088
entail 0.645420491695404
work 1
operate 0.51963871717453
worked 0.4719110429286957
different 1
differing 0.6793351769447327
distinct 0.6211796402931213
type 1
types 0.6495383977890015
subtype 0.54

In [9]:
no_of_cv = 21

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
print(count)
print(idf)

{'lead': 4, 'cause': 1, 'induce': 1, 'ecommerce': 1, 'virtuemart': 1, 'marketplace': 1, 'agency': 1, 'government': 1, 'agencies': 1, 'look': 3, 'looked': 1, 'peek': 1, 'hire': 1, 'recruit': 1, 'supervise': 1, 'web': 14, 'webpage': 3, 'website': 10, 'developer': 17, 'developers': 1, 'programmer': 1, 'join': 2, 'joining': 1, 'joined': 1, 'team': 8, 'teams': 1, 'crew': 1, 'role': 7, 'roles': 1, 'minston': 1, 'involve': 7, 'require': 2, 'entail': 1, 'work': 20, 'operate': 5, 'worked': 1, 'different': 9, 'differing': 1, 'distinct': 1, 'type': 3, 'types': 1, 'subtype': 1, 'client': 9, 'server': 7, 'clients': 1, 'candidate': 1, 'candidates': 1, 'applicant': 1, 'ability': 8, 'capability': 1, 'inability': 1, 'create': 10, 'generate': 8, 'construct': 1, 'implement': 8, 'utilize': 9, 'emulate': 1, 'unique': 3, 'uniquely': 1, 'engage': 3, 'participate': 4, 'engaging': 1, 'intuitive': 3, 'elegant': 1, 'understandable': 1, 'meet': 6, 'satisfy': 1, 'fulfill': 1, 'requirement': 8, 'requirements': 1, '

In [10]:
score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = cvs.loc(0)['skill'][i].split().count(word) + cvs.loc(0)['exp'][i].split().count(word)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass

In [11]:

sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], i))
    
sorted_list.sort(reverse = True)

for s, i in sorted_list:
    if list(cvs)[i] != '.DS_Store':
        print(list(cvs)[i], ':', s)

cv106 : 216.21213593449576
cv118 : 190.73007047316395
cv115 : 174.87460811190587
cv102 : 133.7824659182511
cv116 : 130.87700259483094
cv110 : 122.2947579375003
cv113 : 52.90929792907697
cv100 : 46.08902326103991
cv108 : 39.91719189013247
cv103 : 34.077362830363384
cv112 : 21.46256101431463
cv105 : 18.189834032416208
cv114 : 13.138936604915315
cv101 : 12.904251733957267
cv104 : 12.899188296834469
cv111 : 6.801351228070288
cv107 : 6.184231206153628
cv117 : 5.732873806020064
cv109 : 4.190400703333889
cv11 : 1.040397031346556
cv119 : 0
